In [1]:
import numpy as np
import pandas as pd
import datetime as dt
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

### Read data

In [5]:
pip install openpyxl

     -------------------------------------- 242.1/242.1 kB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
df_=pd.read_excel(r'C:\GITHUB\RFM Analysis\Data\online_retail.xlsx', sheet_name="Year 2009-2010")
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


## Getting to know the variables:
InvoiceNo: The number of the invoice, unique per each purchase. refund invoice numbers contain "C"

StockCode: Unique code per each item

Description: Name of the item

Quantity: The number of items within the invoice

InvoiceDate: Date and time of purchase

UnitPrice: Price of a single item, as of Sterlin

CustomerID: Unique id number per each customer

Country: The country where the custormer is living

### Data Understanding

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


How many countries in df:

In [6]:
df['Country'].nunique()

40

In [7]:
df['Country'].value_counts()

United Kingdom          485852
EIRE                      9670
Germany                   8129
France                    5772
Netherlands               2769
Spain                     1278
Switzerland               1187
Portugal                  1101
Belgium                   1054
Channel Islands            906
Sweden                     902
Italy                      731
Australia                  654
Cyprus                     554
Austria                    537
Greece                     517
United Arab Emirates       432
Denmark                    428
Norway                     369
Finland                    354
Unspecified                310
USA                        244
Japan                      224
Poland                     194
Malta                      172
Lithuania                  154
Singapore                  117
RSA                        111
Bahrain                    107
Canada                      77
Hong Kong                   76
Thailand                    76
Israel  

The most expensive products:

In [9]:
df.sort_values(by='Price', ascending=False).head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
241824,C512770,M,Manual,-1,2010-06-17 16:52:00,25111.09000,17399.00000,United Kingdom
241827,512771,M,Manual,1,2010-06-17 16:53:00,25111.09000,NaN,United Kingdom
320581,C520667,BANK CHARGES,Bank Charges,-1,2010-08-27 13:42:00,18910.69000,NaN,United Kingdom
517953,C537630,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:04:00,13541.33000,NaN,United Kingdom
519294,C537651,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:49:00,13541.33000,NaN,United Kingdom


Number of unique product:

In [10]:
df['Description'].nunique()

4681

Most purchased items:

In [11]:
df.groupby('Description').agg({"Quantity": 'sum'}).sort_values("Quantity", ascending=False).head()

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,57733
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54698
BROCADE RING PURSE,47647
PACK OF 72 RETRO SPOT CAKE CASES,46106
ASSORTED COLOUR BIRD ORNAMENT,44925


Check the number of uniques for StockCode and Description variables:

In [12]:
print(f"Number of uniques in StockCode: {df['StockCode'].nunique()}")
print(f"Number of uniques in Description: {df['Description'].nunique()}")

Number of uniques in StockCode: 4632
Number of uniques in Description: 4681


The values were expected to be equal, so there must be more than one unique value in Description variable for one unique StockCode. Let's check each StockCode value with the corresponding Decription values, get every StockCode that has more than one unique Description in a list form ()

In [14]:
a =df.groupby('StockCode').agg({'Description': "nunique"})
a.reset_index(inplace=True)
a.head()

,StockCode,Description
0,10002,1
1,10080,1
2,10109,1
3,10120,2
4,10125,1


In [21]:
b = list(a.loc[a['Description'] > 1, 'StockCode'])


In [23]:
for dup in b:
    print(f"dup = {dup} {df.loc[df['StockCode'] == dup, 'Description'].unique()}")

dup = 10120 ['DOGGY RUBBER' 'Zebra invcing error']
dup = 16011 [' ANIMAL STICKERS' 'ANIMAL STICKERS']
dup = 16012 ['FOOD/DRINK SPUNGE STICKERS' 'FOOD/DRINK SPONGE STICKERS']
dup = 16235 ['RECYCLED PENCIL WITH RABBIT ERASER' '?']
dup = 17033 ['BROCADE PURSE,SMALL ARCH BUTTON' 'sold as 17003?' 'Sold as 17003?' nan]
dup = 20615 ['BLUE SPOTTY PASSPORT COVER' 'BLUE POLKADOT PASSPORT COVER']
dup = 20652 ['BLUE SPOTTY LUGGAGE TAG ' nan 'BLUE POLKADOT LUGGAGE TAG ']
dup = 20658 ['RED SPOTTY LUGGAGE TAG' 'RED RETROSPOT LUGGAGE TAG']
dup = 20661 ['BLUE SPOTTY PURSE ' 'BLUE POLKADOT PURSE ']
dup = 20665 ['RED SPOTTY PURSE ' 'RED RETROSPOT PURSE ']
dup = 20674 ['GREEN SPOTTY BOWL' 'GREEN POLKADOT BOWL']
dup = 20675 ['BLUE SPOTTY BOWL' 'BLUE POLKADOT BOWL']
dup = 20676 ['RED SPOTTY BOWL' 'RED RETROSPOT BOWL']
dup = 20677 ['PINK SPOTTY BOWL' 'PINK POLKADOT BOWL']
dup = 20679 ['EDWARDIAN PARASOL RED' nan '?']
dup = 20681 ['PINK SPOTTY CHILDS UMBRELLA' 'PINK POLKADOT CHILDRENS UMBRELLA']
dup = 20682 [

There are duplicates in Decription variable, so it would be better to use StockCode

In [4]:
df.loc[df['StockCode'] == 20724, 'Description'].unique()

array(['RED SPOTTY CHARLOTTE BAG', 'RED RETROSPOT CHARLOTTE BAG'],
      dtype=object)

### Data Preperation

Drop na value

In [5]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417534 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      417534 non-null  object        
 1   StockCode    417534 non-null  object        
 2   Description  417534 non-null  object        
 3   Quantity     417534 non-null  int64         
 4   InvoiceDate  417534 non-null  datetime64[ns]
 5   Price        417534 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      417534 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 28.7+ MB


In [6]:
df.describe([0.01, 0.05, 0.1, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Quantity,417534.00000,12.75881,101.22042,-9360.00000,-2.00000,1.00000,1.00000,2.00000,4.00000,12.00000,24.00000,36.00000,144.00000,19152.00000
Price,417534.00000,3.88755,71.13180,0.00000,0.29000,0.42000,0.65000,1.25000,1.95000,3.75000,6.75000,8.50000,14.95000,25111.09000
Customer ID,417534.00000,15360.64548,1680.81132,12346.00000,12435.00000,12725.00000,13042.00000,13983.00000,15311.00000,16799.00000,17706.00000,17913.00000,18196.00000,18287.00000


There are negative values on Quantity variables by refund invoices (invoices containing the letter "C"), reassign df without refund invoices.

In [6]:
df= df[~df["Invoice"].str.contains("C", na=False)]

In [7]:
df.describe([0.01, 0.05, 0.1, 0.5, 0.75, 0.9, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,50%,75%,90%,95%,99%,max
Quantity,515255.00000,10.95663,104.35401,-9600.00000,1.00000,1.00000,1.00000,3.00000,10.00000,24.00000,32.00000,120.00000,19152.00000
Price,515255.00000,3.95637,127.68856,-53594.36000,0.21000,0.42000,0.65000,2.10000,4.21000,7.78000,9.95000,18.95000,25111.09000
Customer ID,407695.00000,15368.50411,1679.79570,12346.00000,12435.00000,12731.00000,13044.00000,15321.00000,16812.00000,17706.00000,17913.00000,18196.00000,18287.00000


In [15]:
df.loc[df["Invoice"].str.contains("C", na=False)]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95000,16321.00000,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65000,16321.00000,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25000,16321.00000,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10000,16321.00000,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95000,16321.00000,Australia
...,...,...,...,...,...,...,...,...
524695,C538123,22956,36 FOIL HEART CAKE CASES,-2,2010-12-09 15:41:00,2.10000,12605.00000,Germany
524696,C538124,M,Manual,-4,2010-12-09 15:43:00,0.50000,15329.00000,United Kingdom
524697,C538124,22699,ROSES REGENCY TEACUP AND SAUCER,-1,2010-12-09 15:43:00,2.95000,15329.00000,United Kingdom
524698,C538124,22423,REGENCY CAKESTAND 3 TIER,-1,2010-12-09 15:43:00,12.75000,15329.00000,United Kingdom


df = df[~df["Invoice"].str.contains("C", na=False)]

Negative values are excluded. We are not removeing outliers like max value on Quantity and Price variables because will be scoring the dataset.

In [8]:
df['TotalPrice'] = df['Quantity'] * df['Price']

In [9]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom,83.40000
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom,100.80000
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom,30.00000


## RFM Metrics

The last day of purchase:

In [11]:
df['InvoiceDate'].max()

Timestamp('2010-12-09 20:01:00')

Assign "today's date" as 2 days after the last date of purchase to make sure that none of the recency values become zero.

In [13]:
today_date = df['InvoiceDate'].max() + dt.timedelta(days=2)
today_date 

Timestamp('2010-12-11 20:01:00')

Create a new df called rfm in order to calculate Recency, Frequency and Monetary values. rfm is grouped by customers and:

- The number of days between today_date and the last purchase date of this customer is Recency

- The number of unique invoices of this customer os Frequency

- The sum of TotalPrice is customer's Monetary

In [15]:
 rfm = df.groupby ('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                        'Invoice': lambda inv: inv.nunique(),
                                        'TotalPrice': lambda price: price.sum()})

In [16]:
rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.00000,166,11,372.86000
12347.00000,4,2,1323.32000
12348.00000,75,1,222.16000
12349.00000,44,3,2671.14000
12351.00000,12,1,300.93000


Rename rfm columns:

In [17]:
rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346.00000,166,11,372.86000
12347.00000,4,2,1323.32000
12348.00000,75,1,222.16000
12349.00000,44,3,2671.14000
12351.00000,12,1,300.93000


Check if there are any zeros in rfm:

In [18]:
rfm.describe([0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Recency,4314.00000,92.26912,96.94348,2.00000,2.00000,4.00000,9.00000,19.00000,54.00000,137.00000,256.00000,305.00000,369.00000,375.00000
Frequency,4314.00000,4.45410,8.16866,1.00000,1.00000,1.00000,1.00000,1.00000,2.00000,5.00000,9.00000,13.00000,31.00000,205.00000
Monetary,4314.00000,2047.28866,8912.52324,0.00000,39.95650,110.43650,155.48500,307.95000,705.55000,1722.80250,3796.59500,6235.12050,20136.78250,349164.35000


### RFM Scores

- The min number of Recency metric means that this customer has just purchased, so the highest score (5) should be given to the lower number of Recency.

The max number of Frequency and Monetary metrics mean that the customer is purchasing frequently and spending more money, so the highest score (5) should be given to the highest Frequency and Monetary values.

In [20]:
rfm["RecencyScore"]= pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
rfm["FrequencyScore"]= pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["MonetaryScore"]= pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])


In [23]:
rfm["RFM_SCORE"] = (rfm['RecencyScore'].astype(str) +
                    rfm['FrequencyScore'].astype(str) + 
                    rfm['MonetaryScore'].astype(str))
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12346.00000,166,11,372.86000,2,5,2,252
12347.00000,4,2,1323.32000,5,2,4,524
12348.00000,75,1,222.16000,2,1,1,211
12349.00000,44,3,2671.14000,3,3,5,335
12351.00000,12,1,300.93000,5,1,2,512


In [25]:
rfm['RFM_SCORE'].max()

'555'

Display some of the customers with the highest scores:

In [28]:
rfm[rfm['RFM_SCORE'] == "555"]

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12415.00000,12,7,19543.84000,5,5,5,555
12431.00000,10,13,4370.52000,5,5,5,555
12471.00000,11,49,20139.74000,5,5,5,555
12472.00000,6,13,11308.48000,5,5,5,555
12474.00000,15,13,5048.66000,5,5,5,555
...,...,...,...,...,...,...,...
18225.00000,2,15,7545.14000,5,5,5,555
18226.00000,15,15,6650.83000,5,5,5,555
18229.00000,3,10,3526.81000,5,5,5,555


### Naming the RFM Scores

The following dict has been made according tho the famous RFM graphic

In [29]:
seg_map = {
    r"[1-2][1-2]": "Hibernating",
    r"[1-2][3-4]": "At Risk",
    r"[1-2]5": "Can't Loose",
    r"3[1-2]": "About to Sleep",
    r"33": "Need Attention",
    r"[3-4][4-5]": "Loyal Customers",
    r"41": "Promising",
    r"51": "New Customers",
    r"[4-5][2-3]": "Potential Loyalists",
    r"5[4-5]": "Champions",
}

We will be using Recency and frequency scores for customer segmentation. We are assuming that a customer who has recently purchased and who is often purchasing should have high RFM scores.

In [30]:
rfm['Segment'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346.00000,166,11,372.86000,2,5,2,252,25
12347.00000,4,2,1323.32000,5,2,4,524,52
12348.00000,75,1,222.16000,2,1,1,211,21
12349.00000,44,3,2671.14000,3,3,5,335,33
12351.00000,12,1,300.93000,5,1,2,512,51


Finally, we will convert the metrics into category names

In [32]:
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346.00000,166,11,372.86000,2,5,2,252,Can't Loose
12347.00000,4,2,1323.32000,5,2,4,524,Potential Loyalists
12348.00000,75,1,222.16000,2,1,1,211,Hibernating
12349.00000,44,3,2671.14000,3,3,5,335,Need Attention
12351.00000,12,1,300.93000,5,1,2,512,New Customers


See the number of customers in each category

In [33]:
rfm['Segment'].value_counts()

Hibernating            1017
Loyal Customers         742
Champions               663
At Risk                 611
Potential Loyalists     517
About to Sleep          343
Need Attention          207
Promising                87
Can't Loose              77
New Customers            50
Name: Segment, dtype: int64

Now we have all the scores for the customers and we have been able to categorize them into 10 groups. We will be using metrics for this process, not scores. We will be focus on the groups that need a better customer relationship and yry to figure out what we can do in order to make that specific segmment purchase more frequently and become loyal.